參考資料：
1. AE in keras: https://blog.keras.io/building-autoencoders-in-keras.html
- Autoencoder 自编码:https://morvanzhou.github.io/tutorials/machine-learning/keras/2-6-autoencoder/
- https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/

In [1]:
import time #時間計算

In [2]:
_startTime = time.time()

In [3]:
import numpy as np
from matplotlib import pyplot as plt
from cv2 import imwrite # 圖片輸出用

In [4]:
# import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input
# from keras import backend as K

E:\Anaconda3\envs\TF-Ke\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from keras.models import load_model

In [6]:
# 結果運算用
from skimage.measure import compare_psnr, compare_ssim 

##### 函數 - 測試用

In [7]:
def CheckType(intVar):
    print("max:",   intVar.max())
    print("min:",   intVar.min())
    print("shape:", intVar.shape)
    print("type:",  intVar.dtype)
    return

##### 函數 - 輸出用

In [8]:
def ResultImgShow(inputImg, modelName, strImgLabel = "TMP"):
    # 輸出參數設置
    img_amount = inputImg.shape[0]
    cols_output = 50
    rows_output = img_amount // cols_output
    if rows_output < 3:
        cols_output = 10
        rows_output = img_amount // cols_output
        if rows_output < 3:
            cols_output = 5
            rows_output = img_amount // cols_output
    if rows_output > cols_output:
        rows_output = cols_output
    # 輸出設置
    img_h = inputImg[0]
    for i in range(1, cols_output):
        img_tmp = inputImg[i]
        img_h = np.hstack((img_h, img_tmp))
    img_v = img_h.copy()
    for j in range(1, rows_output):
        img_h = inputImg[j * cols_output]
        for i in range(1, cols_output):
            img_tmp = inputImg[j * cols_output + i]
            img_h = np.hstack((img_h, img_tmp))
        img_v = np.vstack((img_v, img_h))
        
        
    # 顯示 與 存取
    imwrite(modelName.split(".")[0] +"_"+ strImgLabel+ ".png", img_v)
    plt.imshow(img_v, cmap = "gray")
#     plt.savefig(model_name.split(".")[0] +"_"+ strImgLabel+ ".png")
    plt.show()
    plt.close()
    return

In [9]:
def PSNR_ALL(testData, truthData, strImgLabel = "TMP", boolTest = False):
    psnrSum_test = 0
    for i in range(testData.shape[0]):
        groundTruth   = truthData[i, :,:]
        predictResult = testData[i, :,:]
        psnrTmp = float(compare_psnr(groundTruth, predictResult))
        psnrSum_test += psnrTmp
    psnrSum_test /= testData.shape[0]
#     print("TEST" if boolTest else "TRAIN" ,"PSNR:", psnrSum_test)
    print(strImgLabel, "PSNR:", psnrSum_test)
    return psnrSum_test

In [10]:
def SSIM_ALL(testData, truthData, strImgLabel = "TMP", boolTest = False):
    ssimSum_test = 0
    for i in range(testData.shape[0]):
        groundTruth   = truthData[i, :,:]
        predictResult = testData[i, :,:]
        ssimTmp = float(compare_ssim(groundTruth, predictResult))
        ssimSum_test += ssimTmp
    ssimSum_test /= testData.shape[0]
    print(strImgLabel, "SSIM:", ssimSum_test)
    return ssimSum_test

In [11]:
def DataPreProcess(inputData, img_rows = 28, img_cols = 28):
    outputData = inputData.reshape(inputData.shape[0], img_rows * img_cols)
    outputData = outputData.astype('float32') /255 -0.5
    outputData *= 2
    return outputData

In [12]:
def DataRecovery(inputData, img_rows = 28, img_cols = 28):
    outputData = ((inputData /2)+0.5) * 255
    outputData = outputData.reshape((outputData.shape[0], img_rows, img_cols))
    outputData = outputData.astype('uint8')
    return outputData

# 主程式

#### 參數設置

In [13]:
outputFolder = "output_1212/"

In [14]:
batch_size = 128
# num_classes = 10
epochs = 50
# input image dimensions
img_rows, img_cols = 28, 28

In [15]:
model_name_complex = "model_complex"+"e"+str(epochs)+"_b"+str(batch_size)+".h5"
# model_name_simple  = "model_simple" +"e"+str(epochs)+"_b"+str(batch_size)+".h5"

##### 讀取資料
- 利用完整沒雜訊的資料進行訓練 。
- 只取用圖像，不取標籤。

In [16]:
dataSetName = "mnist.npz"
f = np.load(dataSetName)
x_train_org, x_test_org = f['x_train'], f['x_test']
del f

In [17]:
x_train_org = DataPreProcess(x_train_org)
print(x_train_org.shape[0], 'train samples', "in", x_train_org.shape)
# x_test = DataPreProcess(x_test)
# print(x_test.shape[0], 'tset samples', "in", x_test.shape)

60000 train samples in (60000, 784)


### 模組架設 - 複雜版

In [18]:
# ## 法 1
# autoencoder = Sequential()
# autoencoder.add(Dense(1500, activation='relu', input_shape=(784,)))
# autoencoder.add(Dense(375, activation='relu')) 
# autoencoder.add(Dense(50)) 

# autoencoder.add(Dense(375, activation='relu')) 
# autoencoder.add(Dense(1500, activation='relu')) 
# autoencoder.add(Dense(784, activation='tanh')) 
## 法 2 - 比較有效率
input_img = Input(shape=(784,))
# encoder layers
encoded = Dense(1500, activation='relu')(input_img)
encoded = Dense(375, activation='relu')(encoded)
encoder_output = Dense(50)(encoded)
# decoder layers
decoded = Dense(375, activation='relu')(encoder_output)
decoded = Dense(1500, activation='relu')(decoded)
decoded = Dense(784, activation='tanh')(decoded)
# construct the autoencoder model
autoencoder_complex = Model(input=input_img, output=decoded)

E:\Anaconda3\envs\TF-Ke\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`


#### 模組 - 簡介

In [19]:
autoencoder_complex.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 784)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1500)              1177500   
_________________________________________________________________
dense_2 (Dense)              (None, 375)               562875    
_________________________________________________________________
dense_3 (Dense)              (None, 50)                18800     
_________________________________________________________________
dense_4 (Dense)              (None, 375)               19125     
_________________________________________________________________
dense_5 (Dense)              (None, 1500)              564000    
_________________________________________________________________
dense_6 (Dense)              (None, 784)               1176784   
Total para

#### 模組 - 架設

In [20]:
autoencoder_complex.compile(optimizer='adam', loss='mse')

#### 模組 - 訓練
用完整圖片訓練，完整圖片比對

In [ ]:
history_complex = autoencoder_complex.fit(x_train_org, x_train_org, # 訓練資料、結果比對資料
                    batch_size=batch_size, #
                    epochs=epochs)

Epoch 1/50
60000/60000 [==============================] - 17s 275us/step - loss: 0.0797
Epoch 2/50
60000/60000 [==============================] - 12s 202us/step - loss: 0.0312
Epoch 3/50
60000/60000 [==============================] - 13s 218us/step - loss: 0.0247
Epoch 4/50
60000/60000 [==============================] - 10s 172us/step - loss: 0.0214
Epoch 5/50
60000/60000 [==============================] - 9s 158us/step - loss: 0.0194
Epoch 6/50
60000/60000 [==============================] - 10s 171us/step - loss: 0.0180
Epoch 7/50
60000/60000 [==============================] - 9s 152us/step - loss: 0.0169 0
Epoch 8/50
60000/60000 [==============================] - 11s 183us/step - loss: 0.0161
Epoch 9/50
60000/60000 [==============================] - 7s 121us/step - loss: 0.0154
Epoch 10/50
60000/60000 [==============================] - 6s 100us/step - loss: 0.0149
Epoch 11/50
60000/60000 [==============================] - 6s 101us/step - loss: 0.0144
Epoch 12/50
60000/60000 [========

#### 模組 - 儲存

In [ ]:
model_name_complex = outputFolder + "model_complex"+"e"+str(epochs)+"_b"+str(batch_size) + \
                    ".h5"
#                     "_" + "-".join([str(i) for i in time.localtime()[0:5]]) +\

In [ ]:
autoencoder_complex.save(model_name_complex)

#### 模組 - 使用與輸出

###### 測試資料讀取

In [ ]:
dataSetName = "mnist-pria-awgn_snr=10.npz"
f = np.load(dataSetName)
x_train_noise, x_test_noise = f['x_train'], f['x_test']
del f

In [ ]:
decodeImg_test = autoencoder_complex.predict(x_test_noise)

In [ ]:
decodeImg_test = DataRecovery(decodeImg_test)

In [ ]:
ResultImgShow(decodeImg_test, model_name_complex, strImgLabel = "AFTER")

In [ ]:
ResultImgShow(x_test_noise, model_name_complex, strImgLabel = "BEFORE")
ResultImgShow(x_train_org, model_name_complex, strImgLabel = "ORIGIN")

### 計算結果

In [ ]:
PSNR_ALL(decodeImg_test, x_train_org, strImgLabel = "TEST")
SSIM_ALL(decodeImg_test, x_train_org, strImgLabel = "TEST")

###### 運行時間

In [ ]:
_endTime = time.time()
print("It cost", _endTime - _startTime, "sec.")